In [1]:
import tensorflow as tf

E:\Anaconda3\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 定义一个变量用于计算滑动平均，这个变量的初始值为0.注意这里手动指定变量的类型
# 为float32，因为所有需要计算滑动平均的变量必须是实数型
v1 = tf.Variable(0, dtype=tf.float32)
# 这里的step变量模拟神经网络中迭代的轮数，可以用于动态控制衰减率
step = tf.Variable(0, trainable=False)

In [3]:
# 定义一个滑动平均类（class）。初始化时给定了衰减率（0.99）和控制衰减的变量step
ema = tf.train.ExponentialMovingAverage(0.99, step)

In [4]:
# 定义一个更新变量滑动平均的操作。这里需要给定一个列表，每次执行这个操作时
# 这个列表中的变量都会被更新
maintain_averages_op = ema.apply([v1])

In [5]:
with tf.Session() as sess:
    # 初始化
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    # v1=0 avg(v1)=0
    print(sess.run([v1, ema.average(v1)]))

    # 更新变量v1的取值
    # decay = min(0.99, (1+0)/(10+0)=0.1)=0.1
    # avg(v1) = 0.1 * 0 + 0.9 * 5 = 4.5
    sess.run(tf.assign(v1, 5))
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))

    # 更新step和v1的取值
    # decay = min(0.99, (1+1000)/(10+1000)=0.1)=0.99
    # avg(v1) = 0.99 * 4.5 + 0.01 * 10 = 4.555
    sess.run(tf.assign(step, 10000))
    sess.run(tf.assign(v1, 10))
    sess.run(maintain_averages_op)
    # decay = min(0.99, (1+10000)/(10 + 10000)=0.999)
    # avg(v1) = 0.99 * 4.555 + 0.01 * 10 = 4.60945
    print(sess.run([v1, ema.average(v1)]))

    # 更新一次v1的滑动平均值
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))

[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.555]
[10.0, 4.60945]
